### Задание
Имеются таблицы с данными фильмов.

1 таблица - basics. \
Структура полей и описание: \
tconst (string) - alphanumeric unique identifier of the title \
titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc) \
primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release \
originalTitle (string) - original title, in the original language \
isAdult (boolean) - 0: non-adult title; 1: adult title \
startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year \
endYear (YYYY) – TV Series end year. '\N' for all other title types \
runtimeMinutes – primary runtime of the title, in minutes \
genres (string array) – includes up to three genres associated with the title

2 таблица - ratings. \
Структура полей и описание: \
tconst (string) - alphanumeric unique identifier of the title \
averageRating – weighted average of all the individual user ratings \
numVotes - number of votes the title has received

Нужно показать топ 5 самых высокооцененных жанров фильмов за последние 10, 20, 30 лет (за каждое десятилетие).

In [196]:
# импорт
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window
from funcs.spark_common import spark_read

In [197]:
# список десятилетий, за которые нужно рассчитать топ 5 жанров
range_list = [10, 20, 30]

In [198]:
# инициализация spark
spark = SparkSession \
    .builder \
    .appName("pyspark_imdb_top5") \
    .master("local[2]") \
    .config("spark.driver.memory", "10G") \
    .config("spark.jars", "postgresql-42.7.4.jar") \
    .getOrCreate()

In [199]:
# загружаем данные в датафреймы
df_basics = spark_read(spark, "basics")
df_ratings = spark_read(spark, "ratings")

In [200]:
# join датафреймов и выбор нужных столбцов
df = df_basics \
    .where("titleType = 'movie'") \
    .join(df_ratings, df_basics.tconst == df_ratings.tconst) \
    .select(df_basics.startYear, df_basics.genres, df_ratings.averageRating, df_ratings.numVotes)

# находим максимальный год выпуска фильмов в датафрейме
max_year = df.agg(F.max(F.col("startYear"))).collect()[0][0]

# сплит и разбиение на строки колонки genres
df = df.withColumn("genres", F.explode(F.split("genres", ","))).sort(F.desc("startYear"))

# формируемы выражение case when, чтобы проставить признак десятилетия для каждого фильма
when_str = "\n".join([f"WHEN {max_year}-startYear<={i} THEN 'top5_{i}'" for i in range_list])
sql = f"CASE\n{when_str}\nELSE NULL\nEND"

# проставляем признак десятилетия
df = df.withColumn("top5", F.expr(sql)).dropna(subset="top5")

# окна
w_1 = Window.partitionBy("top5", "genres")
w_2 = Window.partitionBy("top5").orderBy("top5")

# считаем общее количество оценок для каждого жанра в каждом десятилетии
# считаем среднюю оценку для каждого жанра в каждом десятилетии
# считаем максимальное и минимальное количество голосов в каждом десятилетии
# рассчитываем метрику
# дропаем лишние столбцы
# удаляем дубликаты и сортируем

df = df. \
    withColumn("total_votes", F.sum("numVotes").over(w_1)). \
    withColumn("avg_rating", F.mean("averageRating").over(w_1)). \
    withColumn("maxi", F.max("total_votes").over(w_2)) . \
    withColumn("mini", F.min("total_votes").over(w_2)). \
    withColumn("metric", F.round((F.col("avg_rating") * ((F.col("total_votes") - F.col("mini")) / (F.col("maxi") - F.col("mini")))), 2)). \
    drop("startYear", "averageRating", "numVotes", "total_votes", "avg_rating", "maxi", "mini"). \
    distinct(). \
    sort(F.asc("top5"), F.desc("metric"))

# фильтруем топ 5 жанров для каждого десятилетия
df = df. \
    withColumn("id", F.row_number().over(w_2)). \
    where("id <= 5"). \
    drop("metric")

In [201]:
# делаем pivot таблицы, получаем конечный результат
df_result = df.groupBy("id").pivot("top5").agg(F.first("genres"))

# сохраняем результат в паркет
df_result.write.parquet("top5_result.parquet")

In [202]:
df_result.show()

+---+---------+---------+---------+
| id|  top5_10|  top5_20|  top5_30|
+---+---------+---------+---------+
|  1|    Drama|    Drama|    Drama|
|  2|   Action|   Action|   Comedy|
|  3|Adventure|   Comedy|   Action|
|  4|   Comedy|Adventure|Adventure|
|  5|    Crime| Thriller|    Crime|
+---+---------+---------+---------+

